In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats
import math
import numpy as np
from colorama import Fore, Back, Style
from sklearn.linear_model import LinearRegression

## Defining risk attitude and regressor to use

In [ ]:
risk_attitude='Neutral'
regressor='OLS'

## MonteCarlo sampling of priors

In [2]:
N_MC = 10000 # Number of MC samples
FieldLifeTime = 50 # Number of years for production

In [3]:
# Means
OOIP_Mean = 240 # [MMbbl]
E_inf_Phase1_Mean = 0.2  # [Fraction]
Tau_Phase1_Mean = 16 # [Years]
dE_inf_Phase2_Mean = 0.15 # [Fraction]
Tau_Phase2_Mean = 7 # [Years]

# Standard Deviations
OOIP_SD = 35
E_inf_Phase1_SD = 0.05
Tau_Phase1_SD = 2
dE_inf_Phase2_SD = 0.05
Tau_Phase2_SD = 1.5

# Adding some error to the mean and std
ErrorMean4Rate_Pct = 0
ErrorSD4Rate_Pct = 0.10 

##### Correlation coefficients & Covariances

In [4]:
# Production model parameters are correlated to each other. Thefore we can then calculate the Covariance

mu = np.array([OOIP_Mean, E_inf_Phase1_Mean, Tau_Phase1_Mean, dE_inf_Phase2_Mean, Tau_Phase2_Mean])
ExpSigma = np.array([OOIP_SD, E_inf_Phase1_SD, Tau_Phase1_SD, dE_inf_Phase2_SD, Tau_Phase2_SD])

ExpCorrC = np.array([[1, -0.8, 0.16, 0.56, -0.08], [-0.8, 1, 0.2, -0.7, 0.1], [0.16, 0.2, 1, -0.3, -0.2], [0.56, -0.7, -0.3,
1, -0.3], [-0.08, 0.1, -0.2, -0.3, 1]])

# Covariance
cov = ExpCorrC*(ExpSigma.reshape(5,1)*ExpSigma)

##### Random Sampling of the means for the 5 model parameters, as per normal distributions and covariances

In [7]:
# Prior realizations of the model parameters
PriorRealizations=np.random.multivariate_normal(mu, cov, size=N_MC)  # Random Sampling of a multivariate normal distribution 

# Introducing bounds to the samples distributions
for i in range(N_MC): 
    if OOIP_prior[i]<10:
        OOIP_prior[i]=10
    else:
        if OOIP_prior[i]>1000:
            OOIP_prior[i]=1000
        else:
            OOIP_prior[i]=OOIP_prior[i]  
        
E_inf_Phase1_prior = PriorRealizations[:,1]

for i in range(N_MC): 
    if E_inf_Phase1_prior[i]<0.05:
        E_inf_Phase1_prior[i]=0.05
    else:
        if E_inf_Phase1_prior[i]>0.5:
            E_inf_Phase1_prior[i]=0.5
        else:
            E_inf_Phase1_prior[i]=E_inf_Phase1_prior[i]  
            
    
Tau_Phase1_prior = PriorRealizations[:,2]

for i in range(N_MC): 
    if Tau_Phase1_prior[i]<1:
        Tau_Phase1_prior[i]=1
    else:
        if Tau_Phase1_prior[i]>30:
            Tau_Phase1_prior[i]=30
        else:
            Tau_Phase1_prior[i]=Tau_Phase1_prior[i]  
            

dE_inf_Phase2_prior = PriorRealizations[:,3]

for i in range(N_MC): 
    if dE_inf_Phase2_prior[i]<0.01:
        dE_inf_Phase2_prior[i]=0.01
    else:
        if dE_inf_Phase2_prior[i]>0.31:
            dE_inf_Phase2_prior[i]=0.31
        else:
            dE_inf_Phase2_prior[i]=dE_inf_Phase2_prior[i]  
            

Tau_Phase2_prior = PriorRealizations[:,4]

for i in range(N_MC): 
    if Tau_Phase2_prior[i]<1:
        Tau_Phase2_prior[i]=1
    else:
        if Tau_Phase2_prior[i]>13:
            Tau_Phase2_prior[i]=13
        else:
            Tau_Phase2_prior[i]=Tau_Phase2_prior[i]  
            

## Export MC sample as texts for future tests

In [12]:
# Defining desired path
Path="C:/Users/maria/OneDrive - Universitetet i Stavanger/Master thesis/Dataframes_from_Python/" 

np.savetxt(Path+"OOIP_prior", OOIP_prior, delimiter=",")
np.savetxt(Path+"E_inf_Phase1_prior", E_inf_Phase1_prior, delimiter=",")
np.savetxt(Path+"Tau_Phase1_prior", Tau_Phase1_prior, delimiter=",")
np.savetxt(Path+"dE_inf_Phase2_prior", dE_inf_Phase2_prior, delimiter=",")
np.savetxt(Path+"Tau_Phase2_prior", Tau_Phase2_prior, delimiter=",")

In [13]:
# Initializing matrixes
Recovery_Phase1_real_time = np.zeros([N_MC,FieldLifeTime]) 
Recovery_Phase1_real_EndTime = np.zeros([N_MC,1])
Recovery_Phase2_real_time = np.zeros([N_MC,1]) 
Recovery_Phase1n2_real_time = np.zeros([N_MC,FieldLifeTime])
Recovery_Phase1n2_real_time_ShiftTime = np.zeros([N_MC,FieldLifeTime,FieldLifeTime+1])
Rate_Phase1n2_real_time_ShiftTime = np.zeros([N_MC,FieldLifeTime,FieldLifeTime+1])

ER1_0 = 0 # Primary Recovery Factor at t=0

# For calculations below consider 'time' and 't' as positions in time (year) within the production lifetime for each recovery phase (primary and secondary). Assumed to be at the start of the year 

for i_MC in range(N_MC):
    
            
    for time in range(1,FieldLifeTime+1): # This is the time shift in columns. This representes the potential lengths of the Primary Recovery
                              
        Recovery_Phase1_real_time[i_MC,time-1] = ER1_0+np.multiply((E_inf_Phase1_prior[i_MC]-ER1_0),(1-np.exp(-time/Tau_Phase1_prior[i_MC]))) # np.multiply guarantees element-wise multiplication
                
        if time == 1:  
            Recovery_Phase1_real_EndTime[i_MC] = ER1_0 
        else:
            Recovery_Phase1_real_EndTime[i_MC] = Recovery_Phase1_real_time[i_MC,time-2] # Since the ERs are set at the start of each year, the end of the phase 1 should be the start of the previous year
                 
        Recovery_Phase2_real_time[i_MC] = Recovery_Phase1_real_EndTime[i_MC] +np.multiply(dE_inf_Phase2_prior[i_MC],(1-np.exp(-1/Tau_Phase2_prior[i_MC])))
             
        
     # Now we compute the possible combinations of time and t
        
        
        for t in range(1,FieldLifeTime+1): # This is the time shift in rows. This representes the potential positions of the Secondary Recovery
                   
            if t<time:  # Positions in time that fall within the primary recovery. Hence, we store the ER of primary recovery (previously calculated)
                Recovery_Phase1n2_real_time[i_MC,t-1] = Recovery_Phase1_real_time[i_MC,t-1]
                    
            else:
                if t==time: # At this position in time, secondary recovery is just starting in that year. Hence, we store the info of secondary recovery when only lasts 1 year (previously calculated)
                    Recovery_Phase1n2_real_time[i_MC,t-1] = Recovery_Phase2_real_time[i_MC]
                else:
                    # Cases for secondary recovery lasting more than 1 year, which brings now multiple combinations
                    
                    Recovery_Phase1n2_real_time[i_MC,t-1] = Recovery_Phase1_real_EndTime[i_MC] +np.multiply(dE_inf_Phase2_prior[i_MC],(1-np.exp(-(t-time+1)/Tau_Phase2_prior[i_MC])))
                    
            # The matrix below compiles now all the recovery factors at the respective combination of primary and secondary production
            
            Recovery_Phase1n2_real_time_ShiftTime[i_MC,t-1,time-1] = Recovery_Phase1n2_real_time[i_MC,t-1] # This t-variable matrix fills into a complete t,time-variable matrix, for each time
            Recovery_Phase1n2_real_time_ShiftTime[i_MC,t-1,FieldLifeTime+0] = Recovery_Phase1_real_time[i_MC,t-1] # Case of only primary recovery (time = FieldLifeTime), cuando el length del primary recovery os the fieldlifetime
            

# Computation of the flow rates
    
            if t==1: 
                Rate_Phase1n2_real_time_ShiftTime[i_MC,t-1,time-1] = np.multiply(OOIP_prior[i_MC],Recovery_Phase1n2_real_time_ShiftTime[i_MC,t-1,time-1])
                Rate_Phase1n2_real_time_ShiftTime[i_MC,t-1,FieldLifeTime+0] =np.multiply(OOIP_prior[i_MC],Recovery_Phase1n2_real_time_ShiftTime[i_MC,t-1,FieldLifeTime+0])
 
            else: 
                Rate_Phase1n2_real_time_ShiftTime[i_MC,t-1,time-1] = np.multiply(OOIP_prior[i_MC],(Recovery_Phase1n2_real_time_ShiftTime[i_MC,t-1,time-1]-Recovery_Phase1n2_real_time_ShiftTime[i_MC,t-2,time-1]))
                Rate_Phase1n2_real_time_ShiftTime[i_MC,t-1,FieldLifeTime+0] = np.multiply(OOIP_prior[i_MC],(Recovery_Phase1n2_real_time_ShiftTime[i_MC,t-1,FieldLifeTime+0]-Recovery_Phase1n2_real_time_ShiftTime[i_MC,t-2,FieldLifeTime+0]))

            
  

## Calculation of the measurement error and the measured oil production rates

In [15]:
# Adding some error to the mean and std
ErrorMean4Rate_Pct = 0
ErrorSD4Rate_Pct = 0.10 

ErrorSD_Matrix = np.multiply(ErrorSD4Rate_Pct,Rate_Phase1n2_real_time_ShiftTime)  

# Fixing the measurement error for all the tests run with this code
# np.random.RandomState() constructs a random number generator. To be consistent with tests, we create a specific RandomState and draw to pseudorandom numbers in a reproducible way.
random_state=np.random.RandomState(0)

# "Observed" rates (error incorporated) sampling
ObsRate_Phase1n2_real_time_ShiftTime = random_state.normal(Rate_Phase1n2_real_time_ShiftTime,ErrorSD_Matrix)


## Calculation of Cashflow and NPV

In [16]:
CashFlow = np.zeros([N_MC,FieldLifeTime])
DisCashFlow = np.zeros([N_MC,FieldLifeTime])
DisCashFlow_ShiftTime = np.zeros([N_MC,FieldLifeTime,FieldLifeTime+1])
NPV_reals = np.zeros([N_MC,1])
NPVtable_LTPhase1_LTPhase2_real = np.zeros([FieldLifeTime+1,FieldLifeTime+1,N_MC])

### Economic parameters

In [17]:
OilPrice = 50
Capex_Phase1 = 50
Capex_Phase2After1 = 40
Capex_Phase2No1 = 75
Opex_Phase1 = 20
Opex_Phase2 = 30
DisRate = 0.12

In [18]:
# OPEX is deducted every year, depending on which recovery mechanism is being used
# CAPEX is only deducted at the year when the recovery phase is started

for t in range(1,FieldLifeTime+1):
    for time in range(1,FieldLifeTime+2):
        
     
        if t == time: # At this position in time, secondary recovery is just starting in that year
            if time == 1: # First year. Case of secondary recovery without having primary recovery. We subtract CAPEX for secondary recovery at this start
                CashFlow[:,t-1] = np.multiply(OilPrice,ObsRate_Phase1n2_real_time_ShiftTime[:,t-1,time-1]) - Opex_Phase2 - Capex_Phase2No1 
                DisCashFlow[:,t-1] = np.divide(CashFlow[:,t-1],np.power((1+DisRate),t))
            else: # Case of secondary recovery after primary recovery
                CashFlow[:,t-1] = np.multiply(OilPrice,ObsRate_Phase1n2_real_time_ShiftTime[:,t-1,time-1]) - Opex_Phase2 - Capex_Phase2After1 
                DisCashFlow[:,t-1] = np.divide(CashFlow[:,t-1],np.power((1+DisRate),t))
        else:
            if t<time:  # Here are positions in time during primary recovery
                if t == 1: # First year. We subtract CAPEX for primary production at this start
                    CashFlow[:,t-1] = np.multiply(OilPrice,ObsRate_Phase1n2_real_time_ShiftTime[:,t-1,time-1]) - Opex_Phase1 - Capex_Phase1
                    DisCashFlow[:,t-1] = np.divide(CashFlow[:,t-1],np.power((1+DisRate),t))
                else: # Any different to first year. We don't discount CAPEX
                    CashFlow[:,t-1] = np.multiply(OilPrice,ObsRate_Phase1n2_real_time_ShiftTime[:,t-1,time-1]) - Opex_Phase1 
                    DisCashFlow[:,t-1] = np.divide(CashFlow[:,t-1],np.power((1+DisRate),t))
            else: # Here are positions in time after primary recovery. Means we are at secondary recovery (and has already started), hence, we dont discount CAPEX
                CashFlow[:,t-1] = np.multiply(OilPrice,ObsRate_Phase1n2_real_time_ShiftTime[:,t-1,time-1]) - Opex_Phase2  
                DisCashFlow[:,t-1] = np.divide(CashFlow[:,t-1],np.power((1+DisRate),t))

        # This matrix contains the cash flows for each production combination 
        DisCashFlow_ShiftTime[:,t-1,time-1] = DisCashFlow[:,t-1] 
        
       # Now the cumulative NPVs are computed 
        if time == 1:
            NPV_reals = np.sum(DisCashFlow_ShiftTime[:,0:t,time-1],axis=1) 
            NPVtable_LTPhase1_LTPhase2_real[time-1,t+0,:] = NPV_reals[:] 
        
        else:
            if time==FieldLifeTime+1:
                NPV_reals = np.sum(DisCashFlow_ShiftTime[:,0:FieldLifeTime,time-1],axis=1) 
                NPVtable_LTPhase1_LTPhase2_real[time-1,0,:] = NPV_reals[:] 
            else:
                if t==time: # At this position in time, secondary recovery is just starting in that year
                    NPV_reals = np.sum(DisCashFlow_ShiftTime[:,0:time,time-1],axis=1) 
                    NPVtable_LTPhase1_LTPhase2_real[time-1,1,:] = NPV_reals[:]
                else:
                    if t<time:
                        NPV_reals = np.sum(DisCashFlow_ShiftTime[:,0:time-1,time-1],axis=1) 
                        NPVtable_LTPhase1_LTPhase2_real[time-1,0,:] = NPV_reals[:]
                    else:
                        NPV_reals = np.sum(DisCashFlow_ShiftTime[:,0:t,time-1],axis=1) 
                        NPVtable_LTPhase1_LTPhase2_real[time-1,t-time+1,:] = NPV_reals[:]

## Determination of DWPI, and VOPI, DWOI, VWOI

This section determines the Decision Without Information and Decision With Perfect Value Without Information 
and Value of Perfect Information

In [20]:
N_Phase1n2LifeTime = np.sum(range(0,FieldLifeTime+2)) 
NPVvector_1real = np.zeros([N_Phase1n2LifeTime,3])
NPVmatrix_reals = np.zeros([N_Phase1n2LifeTime,N_MC])
Sum_NPVvector_1real = np.zeros([N_Phase1n2LifeTime,1])
meanNPVvector = np.zeros([N_Phase1n2LifeTime,3])
NPV_element_vector = np.zeros([FieldLifeTime+1,FieldLifeTime+1])
Phase1n2LifeTimeTable = np.zeros([N_Phase1n2LifeTime,2])

VWPI_real = np.zeros([N_MC,1])
DWPI_Phase1LifeTime_real = np.zeros([N_MC,1])
DWPI_Phase2LifeTime_real = np.zeros([N_MC,1])

In [22]:
for t in range(1,FieldLifeTime+1):
    for time in range(1,FieldLifeTime+2):
        NPV_element_vector[time-1,t+0] = t
        if time > 1:
            NPV_element_vector[time-1,FieldLifeTime-time+2:FieldLifeTime+1] = FieldLifeTime+1 
        else:
            NPV_element_vector[time-1,t+0] = t
        
([col_NPV_element,row_NPV_element])=np.where(NPV_element_vector[:,:] < FieldLifeTime+1)   
col_NPV_element = col_NPV_element
row_NPV_element =row_NPV_element 

In [24]:
for k in range(1,N_Phase1n2LifeTime+1):
    NPVvector_1real[k-1,0] = col_NPV_element[k-1] 
    NPVvector_1real[k-1,1] = row_NPV_element[k-1]
    meanNPVvector[k-1,0] = col_NPV_element[k-1]
    meanNPVvector[k-1,1] = row_NPV_element[k-1]

for i_MC in range(N_MC):
    for k in range(1,N_Phase1n2LifeTime+1):
                
        NPVvector_1real[k-1,2] =NPVtable_LTPhase1_LTPhase2_real[col_NPV_element[k-1],row_NPV_element[k-1],i_MC]       
        NPVmatrix_reals[k-1,i_MC] = NPVvector_1real[k-1,2]
        
        # For perfect information we identify the maximum NPV , among the t and time combinations, for a single realization        
        VWPI_real[i_MC],DWPI_idx = NPVvector_1real.max(axis=0)[2],NPVvector_1real.argmax(axis=0)[2]                
        DWPI_Phase1LifeTime_real[i_MC] = NPVvector_1real[DWPI_idx,0]
        DWPI_Phase2LifeTime_real[i_MC] = NPVvector_1real[DWPI_idx,1]
        
# This calculates the Mean NPV among the realizations, for each t and time combination
meanNPV = NPVmatrix_reals.mean(axis=1)

for k in range(1,N_Phase1n2LifeTime+1):    
    meanNPVvector[k-1,2] = meanNPV[k-1] 

In [ ]:
# Exercising the decision policy without information
# Choose the maximum average NPV (average NPV has been calculated over all realizations for each combination of t and time) 
EVWOI,DWOI_idx = meanNPVvector.max(axis=0)[2],meanNPVvector.argmax(axis=0)[2] 
DWOI_Phase1LifeTime = col_NPV_element[DWOI_idx] 
DWOI_Phase2LifeTime = row_NPV_element[DWOI_idx]
DWOI_LifeTime = DWOI_Phase1LifeTime + DWOI_Phase2LifeTime
DWPI_LifeTime_real = DWPI_Phase1LifeTime_real + DWPI_Phase2LifeTime_real
Phase1n2LifeTimeTable[:,0] = meanNPVvector[:,0]
Phase1n2LifeTimeTable[:,1] = meanNPVvector[:,1]

# Exercising the decision policy with Perfect information
# Calculate the average of the maximum NPVs that had been calculated for each realization (maximum NPV given by the time and t combinations for each realization) 
EVWPI = np.mean(VWPI_real) 
VOPI = EVWPI - EVWOI

## LSM approach using OLS

This section determines the optimum switch time to secondary recovery, and also the optimum stopping time of secondary recovery.
Regression analysis is performed, as part of the LSM workflow.

### Determining optimal stopping time for each potential switch time
We first determine the optimum stopping time of secondary recovery, for each possible switch time.
The NPV corresponding to the optimal stopping time is recorded into the PathTable

In [28]:
SRDM_Shift_Stop_OptNPV_real = np.zeros([N_MC,1])

PathTable = np.zeros([N_MC,FieldLifeTime+1])
PathTable[:,-1] = NPVtable_LTPhase1_LTPhase2_real[FieldLifeTime+0,0,:]
Phase2_StopTime_real_Phase1LifeTime = np.zeros([N_MC,FieldLifeTime+1])

for k_Shift_time in range(1,FieldLifeTime+1):
    NPV_2Phases_matrix = np.zeros([N_MC,k_Shift_time+1]) 
    
    for k_Stop_time in range(1,k_Shift_time+2):
        # This starts the analysis at year time=50
        NPV_2Phases_matrix[:,k_Stop_time-1] = NPVtable_LTPhase1_LTPhase2_real[FieldLifeTime-k_Shift_time+0,k_Stop_time-1,:] 
           
    for k_StopTime in range(1,k_Shift_time+1):
        if k_StopTime == 1:
            x_data_stop_SRDM = ObsRate_Phase1n2_real_time_ShiftTime[:,0:FieldLifeTime-k_StopTime,FieldLifeTime-k_Shift_time+0] 
            X_stop_SRDM = np.concatenate((np.ones([N_MC,1]), x_data_stop_SRDM), axis=1) 
            y_data_stop_matrix = np.column_stack((NPV_2Phases_matrix[:,k_Shift_time-k_StopTime+0],NPV_2Phases_matrix[:,k_Shift_time-k_StopTime+1]))
            
            # We compare the ENPVS of two consecutive years, and store them in the SRDM_Shift matrix
            # The assesement is done going year by year backwards in time
            #Stop_value = np.max(np.column_stack(((LinearRegression().fit(X_stop_SRDM, NPV_2Phases_matrix[:,k_Shift_time-k_StopTime+0])).predict(X_stop_SRDM),(LinearRegression().fit(X_stop_SRDM, NPV_2Phases_matrix[:,k_Shift_time-k_StopTime+1])).predict(X_stop_SRDM))), axis=1)
            Stop_idx = np.argmax(np.column_stack(((LinearRegression().fit(X_stop_SRDM, NPV_2Phases_matrix[:,k_Shift_time-k_StopTime+0])).predict(X_stop_SRDM),(LinearRegression().fit(X_stop_SRDM, NPV_2Phases_matrix[:,k_Shift_time-k_StopTime+1])).predict(X_stop_SRDM))), axis=1)
              # Stop_value is the Optimum ENPV, however we only need the index at which it sits, to extract the corresponding NPV
                          
            for i_MC in range(N_MC):
                # This is the actual NPV to be stored as per highest ENPV
                # This delivers a single optimum stopping time per realization, which then feeds into the Pathtable
                SRDM_Shift_Stop_OptNPV_real[i_MC,0] = y_data_stop_matrix[i_MC,Stop_idx[i_MC]]
        else:
            x_data_stop_SRDM = ObsRate_Phase1n2_real_time_ShiftTime[:,0:FieldLifeTime-k_StopTime,FieldLifeTime-k_Shift_time+0] 
            X_stop_SRDM = np.concatenate((np.ones([N_MC,1]), x_data_stop_SRDM), axis=1)
            y_data_stop_matrix = np.column_stack((NPV_2Phases_matrix[:,k_Shift_time-k_StopTime+0],SRDM_Shift_Stop_OptNPV_real[:,0]))
            #Stop_value = np.max(np.column_stack(((LinearRegression().fit(X_stop_SRDM, NPV_2Phases_matrix[:,k_Shift_time-k_StopTime+0])).predict(X_stop_SRDM),(LinearRegression().fit(X_stop_SRDM, SRDM_Shift_Stop_OptNPV_real[:,0])).predict(X_stop_SRDM))), axis=1)
            Stop_idx = np.argmax(np.column_stack(((LinearRegression().fit(X_stop_SRDM, NPV_2Phases_matrix[:,k_Shift_time-k_StopTime+0])).predict(X_stop_SRDM),(LinearRegression().fit(X_stop_SRDM, SRDM_Shift_Stop_OptNPV_real[:,0])).predict(X_stop_SRDM))), axis=1)
            # Stop_value is the Optimum ENPV, however we only need the index at which it sits, to extract the corresponding NPV
           
            
            for i_MC in range(N_MC):
                SRDM_Shift_Stop_OptNPV_real[i_MC,0] = y_data_stop_matrix[i_MC,Stop_idx[i_MC]]
                
    for i_MC in range(N_MC):
        # This is the actual NPV to be stored as per highest ENPV, for each realization
        PathTable[i_MC,FieldLifeTime-k_Shift_time+0] = SRDM_Shift_Stop_OptNPV_real[i_MC,0]
        
        for k_Stop_time in range(1,k_Shift_time+2):
            if SRDM_Shift_Stop_OptNPV_real[i_MC,0] == NPV_2Phases_matrix[i_MC,k_Stop_time-1]:
                Phase2_StopTime_real_Phase1LifeTime[i_MC,FieldLifeTime-k_Shift_time+0] =k_Stop_time-1

### Determining the optimum switch time

In [30]:
y_data_continuation = np.zeros([N_MC,1])
y_data_shift = np.zeros([N_MC,1])
y_reg_continuation = np.zeros([N_MC,1])
y_reg_shift = np.zeros([N_MC,1])
x_data = ObsRate_Phase1n2_real_time_ShiftTime[:,:,FieldLifeTime+0]

ValueTable = np.zeros([N_MC,FieldLifeTime+1])
ValueTable[:,-1] = PathTable[:,-1] 
SRDM_DecisionTable = np.zeros([N_MC,FieldLifeTime+1])
SRDM_DecisionTable[:,-1] = 1

for k_Shift_time in range(1,FieldLifeTime+1):
    X = np.concatenate((np.ones([N_MC,1]),x_data[:,0:FieldLifeTime-k_Shift_time]),axis=1) 
    # Now we compare NPVs from continue with primary recovery with switch to secondary recovery, given its optimum stopping time (stored in the Pathtable for each switch year)
    # Then optimum option is compared withn next switch option in time
    y_data_shift = PathTable[:,FieldLifeTime-k_Shift_time+0] 
    y_reg_shift= LinearRegression().fit(X, y_data_shift).predict(X)
    y_data_continuation= ValueTable[:,FieldLifeTime-k_Shift_time+1] 
    y_reg_continuation =LinearRegression().fit(X, y_data_continuation).predict(X)
    ENPVTable_Comparison =np.column_stack((y_reg_shift,y_reg_continuation))
    NPV_SRDM = np.column_stack((y_data_shift, y_data_continuation))
    ENPV_SRDM = np.max(ENPVTable_Comparison,axis=1)
    ENPV_SRDM_idx=np.argmax(ENPVTable_Comparison,axis=1)
        
   
    for i_MC in range(N_MC):
        ValueTable[i_MC,FieldLifeTime-k_Shift_time+0] = NPV_SRDM[i_MC,ENPV_SRDM_idx[i_MC]]
        
        if y_reg_continuation[i_MC] > y_reg_shift[i_MC]:
            SRDM_DecisionTable[i_MC,FieldLifeTime-k_Shift_time+0] = 0
        else:
            SRDM_DecisionTable[i_MC,FieldLifeTime-k_Shift_time+0] = 1
            
        
ShiftValue = PathTable[:,0]
ExpShiftValue = np.mean(ShiftValue)

ContinuationValue = ValueTable[:,0]
ExpContinuationValue = np.mean(ContinuationValue)
EVWII = ExpContinuationValue
VOI = EVWII - EVWOI
VOI_over_EVWOI_pct = VOI/EVWOI*100

DWI_Phase1LifeTime_idx = np.argmax(SRDM_DecisionTable,axis=1)
DWI_Phase1LifeTime = DWI_Phase1LifeTime_idx 

DWI_Phase2LifeTime = np.zeros([N_MC,1])


for i_MC in range(N_MC):
    Phase1LifeTime_idx_real = DWI_Phase1LifeTime_idx[i_MC] 
    DWI_Phase2LifeTime[i_MC] = Phase2_StopTime_real_Phase1LifeTime[i_MC,Phase1LifeTime_idx_real]
    
DWI_LifeTime = np.add(np.array(DWI_Phase1LifeTime),np.array(DWI_Phase2LifeTime).T).T



In [ ]:
# Printing final results

print('EVWOI=',np.round(EVWOI,4))
print('EVWPI=',np.round(EVWPI,4))
print('VOPI=',np.round(VOPI,4))
print('EVWII=',np.round(EVWII,4))
print('VOI=',np.round(VOI,4))
print('DWI_LifeTime=',DWI_LifeTime)
print("Decision without information for Primary recovery length is " ,DWOI_Phase1LifeTime, "years")
print("Decision without information for Secondary recovery length is " ,DWOI_Phase2LifeTime, "years")

In [ ]:
# Exporting final results for further analysis

# Final decisions
np.savetxt(Path+"20230208_Fixed_error_measurement_decisions/20230208_Decisions_"+risk_attitude+"_"+regressor+"_Fixed_error_10000real_Lifetime_Phase1", DWI_Phase1LifeTime, delimiter=",")
np.savetxt(Path+"20230208_Fixed_error_measurement_decisions/20230208_Decisions_"+risk_attitude+"_"+regressor+"_Fixed_error_10000real_Lifetime_Phase2", DWI_Phase2LifeTime, delimiter=",")
np.savetxt(Path+"20230208_Fixed_error_measurement_decisions/20230208_Decisions_"+risk_attitude+"_"+regressor+"_DWOI_Lifetime_both_Phases_1and_2", np.column_stack((DWOI_Phase1LifeTime,DWOI_Phase2LifeTime)),delimiter=",")
np.savetxt(Path+"20230208_Fixed_error_measurement_decisions/20230208_Values_"+risk_attitude+"_"+regressor+"_EVWOI_EVWII_VOI", np.column_stack((EVWOI,EVWII,VOI)),delimiter=",")

# Average DWII , not rounded 
np.savetxt(Path+"20230208_Fixed_error_measurement_decisions/20230208_Decisions_"+risk_attitude+"_"+regressor+"_Average_DWII_Lifetime_both_Phases_1and_2", np.column_stack((np.mean(DWI_Phase1LifeTime),np.mean(DWI_Phase2LifeTime))),delimiter=",")

# Final NPVs
np.savetxt(Path+"20230208_Fixed_error_measurement_decisions/20230208_Final_utilities_"+risk_attitude+"_"+regressor+"_Fixed_error_10000real", ContinuationValue, delimiter=",")